<a href="https://colab.research.google.com/github/sayan0506/Agriculture-project-notes-and-references/blob/main/Potato_Leaf_Classifer_without_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Potato Leaf Classifier with Augmentation


# Import Modules

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile 

import tensorflow as tf

# Dataset Download

Download **Plant Village dataset without augmentation** from the following

Published in: **18-04-2019**

[Plant_leaf_diseases_Mendeley_dataset](https://data.mendeley.com/datasets/tywbtsjrjv/1)


In [2]:
!wget https://data.mendeley.com/public-files/datasets/tywbtsjrjv/files/d5652a28-c1d8-4b76-97f3-72fb80f94efc/file_downloaded

--2021-03-22 15:51:06--  https://data.mendeley.com/public-files/datasets/tywbtsjrjv/files/d5652a28-c1d8-4b76-97f3-72fb80f94efc/file_downloaded
Resolving data.mendeley.com (data.mendeley.com)... 162.159.130.86, 162.159.133.86
Connecting to data.mendeley.com (data.mendeley.com)|162.159.130.86|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com/d29ed9b2-8a5d-4663-8a82-c9174f2c7066 [following]
--2021-03-22 15:51:08--  https://md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com/d29ed9b2-8a5d-4663-8a82-c9174f2c7066
Resolving md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com (md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com)... 52.218.0.72
Connecting to md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com (md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com)|52.218.0.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 868032562 (828M) [application/z

## Unzipping

In [9]:
#main dir
content_dir = '/content/Plant Village Original'

# zip
zip_file = '/content/file_downloaded' 

# 'Plant Village Original' directory to unzip
if not os.path.isdir(content_dir):
  os.mkdir(content_dir)

# unzipping
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
  zip_ref.extractall(content_dir)

# Data Inspection